# Trabalho - INF325 2020

## Equipe 8:

 - Aline Souza Silva (RG: 36.783.233-1)
 - Erik Borges (RG: 15.434.065)
 - Lucas Trinquinato (RG: 50.260.503-0)
 - Paulo Sérgio Marchioreto (27.303.073-5)
 - Renato César Alves de Oliveira (RG: 36.536.287-6)


### Instalando módulo de conexão com o Mongo

In [ ]:
!pip install pymongo

### Instalando módulo para impressão de resultados

In [ ]:
!pip install pprintpp

### Importando módulo de conexão com o Mongo

In [ ]:
from pymongo import MongoClient

### Importando módulo para impressão de resultados

In [ ]:
from pprintpp import pprint

### Conectando na base de dados Mongo

In [ ]:
mongoclient = MongoClient("localhost", 27017)

try:
    mongoclient.server_info()
except:
    mongoclient = MongoClient("mongo", 27017)

### Selecionando base de dados

In [ ]:
db = mongoclient.marketplace_engcomp_unicamp

### Preenchendo base de dados

In [ ]:
%run SeedMongo.ipynb

### Modelo de objetos na base de dados

#### Modelo de documento na collection sellers

``` json

{
    "_id" : ObjectId("5f5e60283189464e09808473"),
    "cpfCnpj" : "00000000000",
    "name" : "Venderdor 0",
    "description" : "Melhor vendedor do Brasil.",
    "address" : {
        "street" : "Rua das Sucesso",
        "number" : 123,
        "district" : "Bairro Santo Antonio",
        "zipcode" : "30360-250",
        "city" : "Belo Horizonte",
        "state" : "Minas Gerais",
        "country" : "Brasil"
    },
    "user" : {
        "email" : "vendedor@gmaill.com",
        "password" : "202cb962ac59075b964b07152d234b70"
    },
    "createdAt" : ISODate("2020-09-13T18:08:40.859Z"),
    "updatedAt" : ISODate("2020-09-13T18:08:40.843Z"),
    "status" : "Actived",
    "rating" : 1,
    "categories" : [ 
        "Categoria A", 
        "Categoria A", 
        "Categoria A"
    ],
    "photo" : "https://image.flaticon.com/icons/svg/2039/2039075.svg",
    "type" : "PF"
}

```

#### Modelo de documento na collection products

``` json

{
    "_id" : ObjectId("5f5e60283189464e09808474"),
    "sku" : "SK000000",
    "title" : "Produto 00",
    "description" : "O melhor do mercado pelo melhor preço",
    "manufactureDetails" : {
        "modelNumber" : "A123X",
        "releaseDate" : ISODate("2020-09-13T18:08:40.851Z")
    },
    "shippingDetails" : {
        "weight" : 350,
        "width" : 10,
        "height" : 10,
        "depth" : 1
    },
    "quantity" : 99,
    "pricing" : {
        "price" : 1000
    },
    "seller" : {
        "cpfCnpj" : "00000000000",
        "name" : "Venderdor 0",
        "description" : "Melhor vendedor do Brasil",
        "rating" : 1,
        "photo" : "https://image.flaticon.com/icons/svg/2039/2039075.svg"
    },
    "productType" : "smartphone",
    "createdAt" : ISODate("2020-09-13T18:08:40.851Z"),
    "updatedAt" : ISODate("2020-09-13T18:08:40.851Z"),
    "status" : "Actived",
    "rating" : 1,
    "categories" : [ 
        "Categoria A"
    ],
    "photos" : [ 
        "https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png", 
        "https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png", 
        "https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png"
    ]
}

```

### Lojistas com maior número de produtos disponíveis na plataforma

In [ ]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$group": {
                "_id": "$seller",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        }
    ]
)

pprint(list(result))

### Lojistas com maior número de produtos disponíveis na plataforma (fazendo join com a tabela sellers para trazer todos os dados do lojista)

In [ ]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$group": {
                "_id": "$seller.cpfCnpj",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        },
        {
            "$lookup": {
                "from": "sellers",
                "localField": "_id",
                "foreignField": "cpfCnpj",
                "as": "seller"
            }
        }
    ]
)

pprint(list(result))

### 5 produtos mais comuns no marketplace

In [ ]:
result = db.products.aggregate([
    {
        "$group": {
            "_id": "$productType",
            "total": {
                    "$sum": 1
                }
        }
    },
    {
        "$sort": {
            "total": -1
        }
    },
    {
        "$limit": 5
    }
])

pprint(list(result))

### 5 produtos mais comuns entre diferentes lojistas (produtos que aparecem em mais lojistas diferentes)

In [ ]:
result = db.products.aggregate([
    {
        "$group": {
            "_id": "$productType",
            "sellers": {
                    "$addToSet": "$seller.cpfCnpj"
                }
        }
    },
    {
        "$project": {
            "_id": 1,
            "qtySellers": { "$size": "$sellers" }
        }
    },
    {
        "$sort": {
            "qtySellers": -1
        }
    },
    {
        "$limit": 5
    }
])

pprint(list(result))

### 10 produtos mais recomendados de acordo com o "rating" do fornecedor e do produto

In [ ]:
result = db.products.find().sort("seller.rating", -1).sort("rating", -1).limit(10)
        
pprint(list(result))

### 5 categorias com mais produtos disponíveis

In [ ]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$unwind": "$categories"
        },
        {
            "$group": {
                "_id": "$categories",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        }
    ]
)

pprint(list(result))